In [1]:
import numpy as np
import pandas as pd
import os
import sqlite3
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import string

##  STEP 0: Import & Inspect

Start by loading both datasets and seeing their structure.

In [2]:
details = pd.read_csv("/kaggle/input/books-dataset-goodreadsmay-2024/Book_Details.csv")
DB_PATH = "/kaggle/input/books-dataset-goodreadsmay-2024/book_reviews.db"

conn = sqlite3.connect(DB_PATH)
reviews = pd.read_sql_query(
    "SELECT * FROM book_reviews", conn
)
conn.close()

In [3]:
#details = details.drop(columns=["Unnamed: 0","cover_image_uri","authorlink",])

In [4]:
details

,Unnamed: 0,book_id,cover_image_uri,book_title,book_details,format,publication_info,authorlink,author,num_pages,genres,num_ratings,num_reviews,average_rating,rating_distribution
0,0,1,https://images-na.ssl-images-amazon.com/images...,Harry Potter and the Half-Blood Prince,"It is the middle of the summer, but there is a...","['652 pages, Paperback']","['First published July 16, 2005']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['652'],"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",3292516,58398,4.58,"{'5': '2,244,154', '4': '775,028', '3': '219,8..."
1,1,2,https://images-na.ssl-images-amazon.com/images...,Harry Potter and the Order of the Phoenix,Harry Potter is about to start his fifth year ...,"['912 pages, Paperback']","['First published June 21, 2003']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['912'],"['Young Adult', 'Fiction', 'Magic', 'Childrens...",3401709,64300,4.50,"{'5': '2,178,760', '4': '856,178', '3': '293,2..."
2,2,3,https://images-na.ssl-images-amazon.com/images...,Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...,"['309 pages, Hardcover']","['First published June 26, 1997']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['309'],"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",10116247,163493,4.47,"{'5': '6,544,542', '4': '2,348,390', '3': '856..."
3,3,5,https://images-na.ssl-images-amazon.com/images...,Harry Potter and the Prisoner of Azkaban,"Harry Potter, along with his best friends, Ron...","['435 pages, Mass Market Paperback']","['First published July 8, 1999']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['435'],"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4215031,84959,4.58,"{'5': '2,892,322', '4': '970,190', '3': '287,7..."
4,4,6,https://images-na.ssl-images-amazon.com/images...,Harry Potter and the Goblet of Fire,It is the summer holidays and soon Harry Potte...,"['734 pages, Paperback']","['First published July 8, 2000']",https://www.goodreads.com/author/show/1077326....,J.K. Rowling,['734'],"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",3718209,69961,4.57,"{'5': '2,500,070', '4': '899,496', '3': '259,7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16220,16220,208877606,https://images-na.ssl-images-amazon.com/images...,Finistère,The 1920s. When wealthy American adolescent Ma...,"['316 pages, Paperback']","['First published January 1, 1951']",https://www.goodreads.com/author/show/151154.F...,Fritz Peters,['316'],"['Fiction', 'LGBT', 'Queer', 'Classics', 'Gay'...",129,22,3.77,"{'5': '36', '4': '41', '3': '41', '2': '8', '1..."
16221,16221,208877648,https://images-na.ssl-images-amazon.com/images...,The Descent,"Set in the 1950s, The Descent asks the questio...","['268 pages, Paperback']","['First published January 1, 1952']",https://www.goodreads.com/author/show/151154.F...,Fritz Peters,['268'],['Fiction'],4,0,4.75,"{'5': '3', '4': '1', '3': '0', '2': '0', '1': ..."
16222,16222,209251042,https://images-na.ssl-images-amazon.com/images...,FDR Unmasked: 73 Years of Medical Cover-ups Th...,THE HIDDEN ILLNESSES THAT SHAPED FDR’S LIFE AN...,"['316 pages, Paperback']","['Published February 27, 2024']",https://www.goodreads.com/author/show/2991326....,Steven Lomazow,['316'],"['Drama', 'Novels', 'Contemporary']",986,11,4.32,"{'5': '469', '4': '431', '3': '38', '2': '33',..."
16223,16223,209441121,https://images-na.ssl-images-amazon.com/images...,Code Yellow in Gretna Green,"In Gateway terminology, a ""Code Yellow"" signif...","['776 pages, Kindle Edition']","['Published March 28, 2024']",https://www.goodreads.com/author/show/22515164...,Linzi Day,['776'],"['Science Fiction Fantasy', 'Fantasy', 'Fictio...",1109,106,4.61,"{'5': '764', '4': '266', '3': '74', '2': '4', ..."


In [5]:
reviews

,book_id,reviewer_id,reviewer_name,likes_on_review,review_content,reviewer_followers,reviewer_total_reviews,review_date,review_rating
0,57094644,114413220,Sofia,582 likes,Just when you thought he was done writing book...,"7,961 followers",234 reviews,"February 24, 2021",None
1,57094644,48328025,megs_bookrack,329 likes,Would you be shocked if I told you this was th...,12.1k followers,"1,802 reviews","March 17, 2024",Rating 5 out of 5
2,57094644,6728955,Mariah,232 likes,So you're telling me Anaisn'ta Daughter of Pos...,490 followers,"1,263 reviews","September 5, 2022",Rating 3 out of 5
3,57094644,101179363,ale (semi hiatus) ‧ ₊˚୨ ♡ ୧ ₊˚,218 likes,"*inserts vine ""anything for you, beyoncé""*upda...","2,709 followers",458 reviews,"June 3, 2021",None
4,2948832,48727754,chan ☆,174 likes,i was excited about this one since it was so w...,55.1k followers,"1,139 reviews","June 9, 2021",Rating 2 out of 5
...,...,...,...,...,...,...,...,...,...
63009,44242501,89281938,Stephen R.,9 likes,This collection of essays from some of the mos...,9 followers,100 reviews,"September 16, 2021",Rating 5 out of 5
63010,44242501,42897455,Michel Ch,4 likes,Proved to me that transhumanism is somewhat ch...,3 followers,17 reviews,"January 6, 2022",Rating 3 out of 5
63011,156787,3359801,Rob,8 likes,"...All things considered,Teranesiais a novel c...",37 followers,521 reviews,"August 9, 2015",Rating 2 out of 5
63012,156787,20201225,Daniel Kenefick,7 likes,"I don’t understand why Greg Egan generally, an...",1 follower,58 reviews,"April 29, 2016",Rating 4 out of 5


## STEP 1: Clean the details table

The details table is mostly metadata, not text-heavy — but it contains several stringified lists and dicts.
You’ll want to normalize them.

### 1.1 Handle stringified lists

#### Processing Genres

In [6]:
import ast

def safe_parse(x):
    if isinstance(x, str):  
        try:
            return ast.literal_eval(x)
        except:
            return []  
    return x 

In [7]:
details['genres'] = details['genres'].apply(safe_parse)

#### Processing Format Column

In [8]:
def extract_format_info(text):
    if not isinstance(text, str):
        return None, None
    
    # Match number + pages + format
    match = re.match(r"(\d+)\s*pages?,\s*(.*)", text.strip())
    if match:
        pages = int(match.group(1))
        fmt = match.group(2).strip()
        return pages, fmt
    
    # Fallback if no page number present
    return None, text.strip()

In [9]:
details['format'] = details['format'].apply(safe_parse)
details['format'] = details['format'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

details[['pages', 'book_format']] = details['format'].apply(
    lambda x: pd.Series(extract_format_info(x))
)


#### Processing Pubishing Info

In [10]:
def extract_date(date):
  if not isinstance(date,str):
    return None
  match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})',date)
  if match:
    return match.group(1)
  return None

In [11]:
details['publication_info'] = details['publication_info'].apply(safe_parse)
details['publication_info'] = details['publication_info'].apply(lambda x: x[0] if isinstance(x,list) and len(x)>0 else None)
details['publication_info'] = details['publication_info'].apply(extract_date)
details['publication_date'] = pd.to_datetime(details['publication_info'], errors='coerce')

#### Processing Pages Column

In [12]:
details['num_pages'] = details['num_pages'].apply(safe_parse)
details['num_pages'] = details['num_pages'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
details['num_pages'] = pd.to_numeric(details['num_pages'], errors='coerce')

In [13]:
def parse_dist(text):
  try:
    return {k:int(v.replace(',', '')) for k, v in text.items()}
  except:
    return None

In [14]:
details['rating_distribution'] = details['rating_distribution'].apply(safe_parse)
details['rating_distribution'] = details['rating_distribution'].apply(parse_dist)

ratings_df = details['rating_distribution'].apply(pd.Series).fillna(0).astype(int)
ratings_df = ratings_df.add_prefix('rating_')
details = pd.concat([details, ratings_df], axis=1)

In [15]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
def clean_text(text):

    text = re.sub(r'http\S+|www\S+', '', text)            # remove URLs
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)                       # remove digits
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop]
    return " ".join(words)

In [17]:
reviews['review_content'] =  reviews['review_content'].apply(lambda x: x.replace('"', '')).apply(clean_text)

reviews = reviews[reviews['review_content'].notna()]

In [18]:
def convert_followers(text):
  if not isinstance(text, str):
    return None
  text = text.replace(' follower','')
  text = text.replace(',','')
  if text.endswith('s'):
      text = text[:-1]
  if 'k' in text:
    return int(float(text.replace('k',''))*1000)
  elif 'm' in text:
    return int(float(text.replace('m',''))*1000000)
  try:
    return int(text)
  except ValueError:
    return None

reviews['reviewer_followers'] = reviews['reviewer_followers'].apply(convert_followers)

In [19]:
def convert_total_reviews(text):
  if not isinstance(text, str):
    return None
  text = text.replace(' review','')
  text = text.replace(',','')
  if text.endswith('s'):
      text = text[:-1]

  if 'k' in text:
    return int(float(text.replace('k',''))*1000)
  elif 'm' in text:
    return int(float(text.replace('m',''))*1000000)
  try:
    return int(text)
  except ValueError:
    return None


reviews['reviewer_total_reviews'] = reviews['reviewer_total_reviews'].apply(convert_total_reviews)

In [20]:
reviews['review_date'] = reviews['review_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [21]:
def convert_likes_on_review(text):
  if not isinstance(text, str):
    return None
  #text = text.replace(' likes','')
  #text = text.replace(',','')
  text = text.split()[0]  


  if 'k' in text:
    return int(float(text.replace('k',''))*1000)
  elif 'm' in text:
    return int(float(text.replace('m',''))*1000000)
  try:
    return int(text)
  except ValueError:
    return None


reviews['likes_on_review'] = reviews['likes_on_review'].apply(convert_likes_on_review)

In [22]:
def extract_rating(st):
  if pd.isna(st):
    return np.nan
  lis = st.split()
  return int(lis[1])


reviews['review_rating'] = reviews['review_rating'].apply(extract_rating)

In [23]:
details.to_csv('details.csv', index=False)
reviews.to_csv('reviews.csv',index=False)